>Hyperparameters being tuned


>Number of hidden layers

>Number of neurons (units) per layer

>Activation function for each layer

>Dropout rate per layer

>Regularization type (none, l1, l2, or l1_l2)

>Regularization strength (value)

>Optimizer type (adam, rmsprop, or sgd)

>Learning rate

In [10]:


import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, ReLU
from tensorflow.keras.optimizers import Adam


X, y = make_moons(n_samples=1000, noise=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers, optimizers
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# === 1. Build Model Function for Keras Tuner ===
def build_model(hp):
    model = Sequential()

    # Regularization type and value
    reg_type = hp.Choice("reg_type", ["none", "l1", "l2", "l1_l2"])
    reg_value = hp.Choice("reg_value", [1e-5, 1e-4, 1e-3])

    if reg_type == "l1":
        reg = regularizers.l1(reg_value)
    elif reg_type == "l2":
        reg = regularizers.l2(reg_value)
    elif reg_type == "l1_l2":
        reg = regularizers.l1_l2(l1=reg_value, l2=reg_value)
    else:
        reg = None

    # Hidden layers
    num_layers = hp.Int("num_layers", min_value=2, max_value=5)
    for i in range(num_layers):
        units = hp.Int(f"units_{i}", min_value=16, max_value=128, step=16)
        activation = hp.Choice(f"activation_{i}", ["relu", "tanh", "elu"])

        if i == 0:
            model.add(Dense(units, activation=activation, input_dim=2, kernel_regularizer=reg))
        else:
            model.add(Dense(units, activation=activation, kernel_regularizer=reg))

        model.add(BatchNormalization())
        model.add(Dropout(hp.Float(f"dropout_{i}", 0.1, 0.5, step=0.1)))

    # Output layer
    model.add(Dense(1, activation="sigmoid"))

    # Optimizer and learning rate
    optimizer_choice = hp.Choice("optimizer", ["adam", "rmsprop", "sgd"])
    learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-2, sampling="log")

    if optimizer_choice == "adam":
        optimizer = optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == "rmsprop":
        optimizer = optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = optimizers.SGD(learning_rate=learning_rate, momentum=0.9)

    # Compile model
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )

    return model


# === 2. Create Tuner ===
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=10,                  # Try up to 30 combinations
    executions_per_trial=1,
    directory="tuner_results",
    project_name="classification_tuning"
)


# === 3. Define Callbacks ===
early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6)


# === 4. Run Hyperparameter Search ===
tuner.search(
    X_train, y_train,
    epochs=100,
    validation_split=0.2,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


# === 5. Get the Best Model ===
best_model = tuner.get_best_models(num_models=1)[0]
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\n✅ Best Hyperparameters Found:")
for key, value in best_hps.values.items():
    print(f"{key}: {value}")

# === 6. Evaluate on Test Set ===
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=0)
print(f"\n🎯 Test Accuracy: {test_acc:.4f}")
print(f"🧾 Test Loss: {test_loss:.4f}")

# === 7. (Optional) Retrain on Full Training Data ===
print("\n🚀 Retraining best model on full training data...")
history = best_model.fit(
    X_train, y_train,
    epochs=100,
    validation_data=(X_test, y_test),
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# === 8. Final Evaluation ===
final_loss, final_acc = best_model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Final Test Accuracy: {final_acc:.4f}")


Trial 10 Complete [00h 00m 02s]
val_accuracy: 0.46875

Best val_accuracy So Far: 0.981249988079071
Total elapsed time: 00h 00m 34s

✅ Best Hyperparameters Found:
reg_type: none
reg_value: 0.0001
num_layers: 4
units_0: 16
activation_0: relu
dropout_0: 0.5
units_1: 96
activation_1: tanh
dropout_1: 0.2
optimizer: adam
lr: 0.003818588357106265
units_2: 32
activation_2: elu
dropout_2: 0.5
units_3: 16
activation_3: relu
dropout_3: 0.1

🎯 Test Accuracy: 0.9700
🧾 Test Loss: 0.1083

🚀 Retraining best model on full training data...
Epoch 1/100


/Users/owner/Desktop/deep-learning/deep_learning/lib/python3.13/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9112 - loss: 0.2311 - val_accuracy: 0.9750 - val_loss: 0.0935 - learning_rate: 0.0038
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9125 - loss: 0.2185 - val_accuracy: 0.9750 - val_loss: 0.0896 - learning_rate: 0.0038
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9162 - loss: 0.2309 - val_accuracy: 0.9650 - val_loss: 0.0920 - learning_rate: 0.0038
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9050 - loss: 0.2370 - val_accuracy: 0.9750 - val_loss: 0.0757 - learning_rate: 0.0038
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8925 - loss: 0.2392 - val_accuracy: 0.9750 - val_loss: 0.0752 - learning_rate: 0.0038
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9162 - loss: 0.2201 - val_accuracy: 0.9700 - val_loss: 0.0685 - learning_rate: 0.0038
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9162 - loss: 0.2306 - val_accuracy: 0.970